Connected to ssg-thefan-analysis-py3.13 (Python 3.13.7)

In [ ]:
import os
import re
import sys
import sqlite3
import requests
import numpy as np
import folium

import pandas as pd
pd.set_option("display.max_rows", None)

import geopandas as gpd
import matplotlib.pyplot as plt

from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from math import floor
from pathlib import Path

from fandu.mapping_utils import get_boundary_map
from fandu.geo_utils import get_newest_path

from loguru import logger
# Configure loguru to only log to stderr (console)
logger.remove()  # remove default handler
logger.add(sys.__stderr__, level="INFO", format="{level: <8} | {name}:{function}:{line} - {message}" )  # use the original stderr, not Jupyter’s proxy

## Set up itables
import itables
from itables import show
# 🔧 Sensible global defaults for itables
itables.options.maxBytes = 0                        # show full content in each cell
itables.options.classes = ["display", "compact"]    # compact, clean look
itables.options.lengthMenu = [10, 25, 50, 100]      # page length menu
itables.options.pageLength = 25
itables.options.buttons = ["copy", "csv", "print"]
itables.options.scrollX = True                      # allow horizontal scroll if needed
itables.options.scrollY = True                      # allow horizontal scroll if needed
itables.options.ordering = True

## Set up duckdb

import duckdb
con = duckdb.connect()
x = con.execute("INSTALL spatial; LOAD spatial;")

def show_result_set( query, **kwargs ):
    print('::: {.column-screen-inset style="font-size:0.7em"}')
    df = con.execute( query ).fetch_df()
    show(df,**kwargs)
    print(':::')


# Load and filter files

precious_folder = Path("../precious/")

address_file_root = "Addresses"
boundary_file_root = "Civic_Associations"
boundary_selector = "Fan District Association"

boundary_path = get_newest_path( precious_folder,boundary_file_root )
m, boundary_layer, boundary_shape = get_boundary_map( boundary_path,boundary_selector )

addresses_path = get_newest_path( precious_folder,address_file_root)
addresses_gpd = gpd.read_file( addresses_path )
addresses_gpd = addresses_gpd.to_crs( boundary_shape.crs )


# columns to drop:
shared_drops = ['OBJECTID','CreatedBy','CreatedDate','EditBy','EditDate']

addresses_gpd = addresses_gpd.drop( columns = shared_drops )
boundary_shape = boundary_shape.drop( columns = shared_drops )




filtered_addresses = gpd.sjoin( addresses_gpd, boundary_shape, predicate="within", how="inner")


# Drop columns created from spatial join
shared_drops = ["GlobalID","index_right","AdoptionDate","ChangeDate","Shape__Area","Shape__Length"]

filtered_addresses = filtered_addresses.drop( columns = shared_drops )


gdf = filtered_addresses

# Trim and normalize white space
gdf = gdf.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
gdf = gdf.apply(lambda col: col.str.strip().str.replace(r"\s+", " ", regex=True) 
                if col.dtype == "object" else col)

# Make sure values are integers, or compare as strings consistently
gdf.loc[~gdf["ZipCode"].isin(['23220', '23284']), "ZipCode"] = '23220'

# Clean up double spaces
gdf["AddressLabel"] = (
    gdf["AddressLabel"]
    .astype(str)
    # Replace non-breaking spaces with regular spaces
    .str.replace(u"\u00A0", " ", regex=False)
    # Remove other invisible/control characters
    .str.replace(r"[\x00-\x1F\x7F-\x9F]", "", regex=True)
    # Collapse multiple spaces (including tabs/newlines) into one
    .str.replace(r"\s+", " ", regex=True)
    # Trim leading/trailing whitespace
    .str.strip()
)

gdf["HasDoubleSpaces"] = gdf["AddressLabel"].astype(str).str.contains(r"\s{2,}")
gdf["AddressLen"] = gdf["AddressLabel"].astype(str).str.len()
bad_rows = gdf[gdf["HasDoubleSpaces"]]
print(bad_rows[["AddressLabel", "AddressLen"]])



## New columns
gdf["AddressBase"] = (
    gdf["BuildingNumber"].fillna("").astype(str).str.strip() + " " +
    gdf["StreetDirection"].fillna("").astype(str).str.strip() + " " +
    gdf["StreetName"].fillna("").astype(str).str.strip() + " " +
    gdf["StreetType"].fillna("").astype(str).str.strip()
).str.replace(r"\s+", " ", regex=True).str.strip()

def make_extension(row):
    parts = []
    # only include UnitType if it's not None, empty, or "None"
    if row["UnitType"] not in [None, "", "None"]:
        parts.append(str(row["UnitType"]).strip())
    # only include UnitValue if it's not None or empty
    if row["UnitValue"] not in [None, ""]:
        parts.append(str(row["UnitValue"]).strip())
    return " ".join(parts)

gdf["AddressExtension"] = gdf.apply(make_extension, axis=1)



# If AddressExtension is empty but ExtensionWithUnit has value → copy over
gdf.loc[
    (gdf["AddressExtension"].str.len() == 0) & (gdf["ExtensionWithUnit"].str.len() > 0),
    "AddressExtension"
] = gdf["ExtensionWithUnit"]

# If ExtensionWithUnit is empty but AddressExtension has value → copy over
gdf.loc[
    (gdf["ExtensionWithUnit"].str.len() == 0) & (gdf["AddressExtension"].str.len() > 0),
    "ExtensionWithUnit"
] = gdf["AddressExtension"]



# Condition: AddressLabel contains both "REAR" and "APT"
mask = gdf["AddressLabel"].str.contains("REAR APT", case=False, na=False)

# Remove "REAR" from AddressLabel
gdf.loc[mask, "AddressLabel"] = gdf.loc[mask, "AddressLabel"].str.replace(
    "REAR APT", "APT", case=False, regex=False
).str.strip()

# Set ExtensionWithUnit = AddressExtension for these rows
gdf.loc[mask, "ExtensionWithUnit"] = gdf.loc[mask, "AddressExtension"]


gdf.loc[gdf["AddressBase"] == "425 Strawberry St", "AddressLabel"] = (
    gdf["AddressBase"] + " " + gdf["AddressExtension"].fillna("")
).str.strip()


feature_name = address_file_root + "_in_fan"
gdf.to_parquet( f"{feature_name}.parquet",engine="pyarrow")


x = con.execute("CREATE OR REPLACE TABLE addresses AS SELECT * FROM 'Addresses_in_fan.parquet';")

show_result_set("""
select 
  ZipCode,
  count(*)
from
  addresses
group by
  ZipCode
order by
  ZipCode
""",pageLength=10)

show_result_set("""
select 
  *
from
  addresses
where
  not ZipCode in ('23220','23284')
order by
  StreetName,AddressLabel

""",pageLength=10)

show_result_set("""
select StreetName from addresses group by StreetName order by StreetName
""",pageLength=10)

show_result_set("""
select StreetType from addresses group by StreetType order by StreetType
""",pageLength=10)

show_result_set("""
WITH StreetTypes AS (
    SELECT DISTINCT
           regexp_extract(AddressBase, '[^ ]+$') AS StreetType
    FROM addresses
    WHERE AddressBase IS NOT NULL
),
AddressWithType AS (
    SELECT AddressBase,
           regexp_extract(AddressBase, '[^ ]+$') AS LastWord,
           StreetType
    FROM addresses
    WHERE AddressBase IS NOT NULL
)
SELECT a.AddressBase,LastWord, StreetType
FROM AddressWithType a
where LastWord<>StreetType
""",pageLength=10)

show_result_set("""
SELECT *
FROM addresses
WHERE AddressLabel NOT LIKE AddressBase || '%';
""",pageLength=10)

show_result_set("""
SELECT 
  '"' || AddressLabel || '"' as AddressLabel,
  '"' || AddressBase || '"' as AddressBase,
  '"' || AddressExtension || '"' as AddressExtension,
  '"' || ExtensionWithUnit || '"' as ExtensionWithUnit,
  len(addresslabel) as LabelLen,
  len(addressbase) as BaseLen,
  len(addressextension) as ExtensionLen
FROM addresses
WHERE upper(AddressLabel) NOT LIKE ('% ' || upper(AddressExtension) || '%')
  AND AddressExtension IS NOT NULL 
  AND AddressExtension <> '';
""",pageLength=10)

show_result_set("""
SELECT 
  AddressLabel,
  ExtensionWithUnit,
  AddressExtension,
  length(AddressExtension),
  length(ExtensionWithUnit)
FROM addresses
WHERE
  length(AddressExtension)>0
  and length(ExtensionWithUnit)>0
  and upper(AddressExtension) <> upper(ExtensionWithUnit)
""",pageLength=10)

show_result_set("""
SELECT 
  '"' || AddressLabel || '"' as AddressLabel,
  '"' || AddressBase || '"' as AddressBase,
  '"' || AddressExtension || '"' as AddressExtension,
  hex(AddressLabel),
  len(AddressLabel),
  len(addressbase),
  len(addressextension)
FROM addresses
WHERE (upper(AddressLabel) <> upper(AddressBase || ' ' || AddressExtension) )
  AND AddressExtension IS NOT NULL 
  AND AddressExtension <> ''
""",pageLength=10)

show_result_set("""
WITH unittype_cte AS (
    SELECT unittype
    FROM addresses
    WHERE not unittype in ('None','')
    GROUP BY unittype
)
SELECT * from unittype_cte
""",pageLength=10)

show_result_set("""
WITH unittype_cte AS (
    SELECT unittype
    FROM addresses
    WHERE unittype <> 'None'
    GROUP BY unittype
)
SELECT
    a.AddressLabel,
    a.AddressBase,
    a.AddressExtension,
    a.ExtensionWithUnit,
    a.UnitType,
    a.UnitValue
FROM addresses a
WHERE 
  ((a.ExtensionWithUnit IS NULL OR a.ExtensionWithUnit = '')
    or (a.UnitType is NULL or a.UnitType='' or a.UnitType='None') )
  and regexp_matches(AddressLabel, '\\bSte\\b');
""",pageLength=10)

df = con.execute("""
select * from addresses
""").fetch_df().drop(columns=["geometry","__index_level_0__"])
show(df,pageLength=10)

NameError: name 'tooltip_field' is not defined

In [ ]:
import os
import re
import sys
import sqlite3
import requests
import numpy as np
import folium

import pandas as pd
pd.set_option("display.max_rows", None)

import geopandas as gpd
import matplotlib.pyplot as plt

from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from math import floor
from pathlib import Path

from fandu.mapping_utils import get_boundary_map
from fandu.geo_utils import get_newest_path

from loguru import logger
# Configure loguru to only log to stderr (console)
logger.remove()  # remove default handler
logger.add(sys.__stderr__, level="INFO", format="{level: <8} | {name}:{function}:{line} - {message}" )  # use the original stderr, not Jupyter’s proxy

## Set up itables
import itables
from itables import show
# 🔧 Sensible global defaults for itables
itables.options.maxBytes = 0                        # show full content in each cell
itables.options.classes = ["display", "compact"]    # compact, clean look
itables.options.lengthMenu = [10, 25, 50, 100]      # page length menu
itables.options.pageLength = 25
itables.options.buttons = ["copy", "csv", "print"]
itables.options.scrollX = True                      # allow horizontal scroll if needed
itables.options.scrollY = True                      # allow horizontal scroll if needed
itables.options.ordering = True

## Set up duckdb

import duckdb
con = duckdb.connect()
x = con.execute("INSTALL spatial; LOAD spatial;")

def show_result_set( query, **kwargs ):
    print('::: {.column-screen-inset style="font-size:0.7em"}')
    df = con.execute( query ).fetch_df()
    show(df,**kwargs)
    print(':::')


# Load and filter files

precious_folder = Path("../precious/")

address_file_root = "Addresses"
boundary_file_root = "Civic_Associations"
boundary_selector = "Fan District Association"

boundary_path = get_newest_path( precious_folder,boundary_file_root )
m, boundary_layer, boundary_shape = get_boundary_map( boundary_path,boundary_selector )

addresses_path = get_newest_path( precious_folder,address_file_root)
addresses_gpd = gpd.read_file( addresses_path )
addresses_gpd = addresses_gpd.to_crs( boundary_shape.crs )


# columns to drop:
shared_drops = ['OBJECTID','CreatedBy','CreatedDate','EditBy','EditDate']

addresses_gpd = addresses_gpd.drop( columns = shared_drops )
boundary_shape = boundary_shape.drop( columns = shared_drops )




filtered_addresses = gpd.sjoin( addresses_gpd, boundary_shape, predicate="within", how="inner")


# Drop columns created from spatial join
shared_drops = ["GlobalID","index_right","AdoptionDate","ChangeDate","Shape__Area","Shape__Length"]

filtered_addresses = filtered_addresses.drop( columns = shared_drops )


gdf = filtered_addresses

# Trim and normalize white space
gdf = gdf.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
gdf = gdf.apply(lambda col: col.str.strip().str.replace(r"\s+", " ", regex=True) 
                if col.dtype == "object" else col)

# Make sure values are integers, or compare as strings consistently
gdf.loc[~gdf["ZipCode"].isin(['23220', '23284']), "ZipCode"] = '23220'

# Clean up double spaces
gdf["AddressLabel"] = (
    gdf["AddressLabel"]
    .astype(str)
    # Replace non-breaking spaces with regular spaces
    .str.replace(u"\u00A0", " ", regex=False)
    # Remove other invisible/control characters
    .str.replace(r"[\x00-\x1F\x7F-\x9F]", "", regex=True)
    # Collapse multiple spaces (including tabs/newlines) into one
    .str.replace(r"\s+", " ", regex=True)
    # Trim leading/trailing whitespace
    .str.strip()
)

gdf["HasDoubleSpaces"] = gdf["AddressLabel"].astype(str).str.contains(r"\s{2,}")
gdf["AddressLen"] = gdf["AddressLabel"].astype(str).str.len()
bad_rows = gdf[gdf["HasDoubleSpaces"]]
print(bad_rows[["AddressLabel", "AddressLen"]])



## New columns
gdf["AddressBase"] = (
    gdf["BuildingNumber"].fillna("").astype(str).str.strip() + " " +
    gdf["StreetDirection"].fillna("").astype(str).str.strip() + " " +
    gdf["StreetName"].fillna("").astype(str).str.strip() + " " +
    gdf["StreetType"].fillna("").astype(str).str.strip()
).str.replace(r"\s+", " ", regex=True).str.strip()

def make_extension(row):
    parts = []
    # only include UnitType if it's not None, empty, or "None"
    if row["UnitType"] not in [None, "", "None"]:
        parts.append(str(row["UnitType"]).strip())
    # only include UnitValue if it's not None or empty
    if row["UnitValue"] not in [None, ""]:
        parts.append(str(row["UnitValue"]).strip())
    return " ".join(parts)

gdf["AddressExtension"] = gdf.apply(make_extension, axis=1)



# If AddressExtension is empty but ExtensionWithUnit has value → copy over
gdf.loc[
    (gdf["AddressExtension"].str.len() == 0) & (gdf["ExtensionWithUnit"].str.len() > 0),
    "AddressExtension"
] = gdf["ExtensionWithUnit"]

# If ExtensionWithUnit is empty but AddressExtension has value → copy over
gdf.loc[
    (gdf["ExtensionWithUnit"].str.len() == 0) & (gdf["AddressExtension"].str.len() > 0),
    "ExtensionWithUnit"
] = gdf["AddressExtension"]



# Condition: AddressLabel contains both "REAR" and "APT"
mask = gdf["AddressLabel"].str.contains("REAR APT", case=False, na=False)

# Remove "REAR" from AddressLabel
gdf.loc[mask, "AddressLabel"] = gdf.loc[mask, "AddressLabel"].str.replace(
    "REAR APT", "APT", case=False, regex=False
).str.strip()

# Set ExtensionWithUnit = AddressExtension for these rows
gdf.loc[mask, "ExtensionWithUnit"] = gdf.loc[mask, "AddressExtension"]


gdf.loc[gdf["AddressBase"] == "425 Strawberry St", "AddressLabel"] = (
    gdf["AddressBase"] + " " + gdf["AddressExtension"].fillna("")
).str.strip()


feature_name = address_file_root + "_in_fan"
gdf.to_parquet( f"{feature_name}.parquet",engine="pyarrow")


x = con.execute("CREATE OR REPLACE TABLE addresses AS SELECT * FROM 'Addresses_in_fan.parquet';")

show_result_set("""
select 
  ZipCode,
  count(*)
from
  addresses
group by
  ZipCode
order by
  ZipCode
""",pageLength=10)

show_result_set("""
select 
  *
from
  addresses
where
  not ZipCode in ('23220','23284')
order by
  StreetName,AddressLabel

""",pageLength=10)

show_result_set("""
select StreetName from addresses group by StreetName order by StreetName
""",pageLength=10)

show_result_set("""
select StreetType from addresses group by StreetType order by StreetType
""",pageLength=10)

show_result_set("""
WITH StreetTypes AS (
    SELECT DISTINCT
           regexp_extract(AddressBase, '[^ ]+$') AS StreetType
    FROM addresses
    WHERE AddressBase IS NOT NULL
),
AddressWithType AS (
    SELECT AddressBase,
           regexp_extract(AddressBase, '[^ ]+$') AS LastWord,
           StreetType
    FROM addresses
    WHERE AddressBase IS NOT NULL
)
SELECT a.AddressBase,LastWord, StreetType
FROM AddressWithType a
where LastWord<>StreetType
""",pageLength=10)

show_result_set("""
SELECT *
FROM addresses
WHERE AddressLabel NOT LIKE AddressBase || '%';
""",pageLength=10)

show_result_set("""
SELECT 
  '"' || AddressLabel || '"' as AddressLabel,
  '"' || AddressBase || '"' as AddressBase,
  '"' || AddressExtension || '"' as AddressExtension,
  '"' || ExtensionWithUnit || '"' as ExtensionWithUnit,
  len(addresslabel) as LabelLen,
  len(addressbase) as BaseLen,
  len(addressextension) as ExtensionLen
FROM addresses
WHERE upper(AddressLabel) NOT LIKE ('% ' || upper(AddressExtension) || '%')
  AND AddressExtension IS NOT NULL 
  AND AddressExtension <> '';
""",pageLength=10)

show_result_set("""
SELECT 
  AddressLabel,
  ExtensionWithUnit,
  AddressExtension,
  length(AddressExtension),
  length(ExtensionWithUnit)
FROM addresses
WHERE
  length(AddressExtension)>0
  and length(ExtensionWithUnit)>0
  and upper(AddressExtension) <> upper(ExtensionWithUnit)
""",pageLength=10)

show_result_set("""
SELECT 
  '"' || AddressLabel || '"' as AddressLabel,
  '"' || AddressBase || '"' as AddressBase,
  '"' || AddressExtension || '"' as AddressExtension,
  hex(AddressLabel),
  len(AddressLabel),
  len(addressbase),
  len(addressextension)
FROM addresses
WHERE (upper(AddressLabel) <> upper(AddressBase || ' ' || AddressExtension) )
  AND AddressExtension IS NOT NULL 
  AND AddressExtension <> ''
""",pageLength=10)

show_result_set("""
WITH unittype_cte AS (
    SELECT unittype
    FROM addresses
    WHERE not unittype in ('None','')
    GROUP BY unittype
)
SELECT * from unittype_cte
""",pageLength=10)

show_result_set("""
WITH unittype_cte AS (
    SELECT unittype
    FROM addresses
    WHERE unittype <> 'None'
    GROUP BY unittype
)
SELECT
    a.AddressLabel,
    a.AddressBase,
    a.AddressExtension,
    a.ExtensionWithUnit,
    a.UnitType,
    a.UnitValue
FROM addresses a
WHERE 
  ((a.ExtensionWithUnit IS NULL OR a.ExtensionWithUnit = '')
    or (a.UnitType is NULL or a.UnitType='' or a.UnitType='None') )
  and regexp_matches(AddressLabel, '\\bSte\\b');
""",pageLength=10)

df = con.execute("""
select * from addresses
""").fetch_df().drop(columns=["geometry","__index_level_0__"])
show(df,pageLength=10)

NameError: name 'tooltip_field' is not defined